In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

import sys
from pathlib import Path

project_root = Path.cwd().parent.parent
sys.path.append(str(project_root))
from db.ohlcv.queries import load_ohlcv
from db.ohlcv.utils import get_ibex_tickers
from models.trees.features import safe_build_features

In [3]:
micro = get_ibex_tickers()
df_micro = load_ohlcv(micro)
# Some days have 0 volume e.g on Christmas (API errors)
df_micro = df_micro[df_micro["volume"] > 0]
print(df_micro.shape)
df_micro.head(1)

(126939, 7)


,ticker,date,open,high,low,close,volume
0,ACS.MC,2006-01-02,8.884718,9.038241,8.835722,8.966379,772525.0


In [ ]:
df_final = safe_build_features(df_micro,horizon=1)
df_final = df_final.sort_values("date").reset_index(drop=True)
remove_cols = ["ticker","date","open","high","low","close", "volume","target","future_log_ret"]
X = df_final.drop(columns=remove_cols)
X = X.replace([np.inf, -np.inf], np.nan)
mask = X.notna().all(axis=1)
X = X[mask]
y = df_final["target"]
y = y[mask]
X.shape

(125501, 28)

In [25]:
df_final = safe_build_features(df_micro,horizon=5)
df_final = df_final.sort_values("date").reset_index(drop=True)
remove_cols = ["ticker","date","open","high","low","close", "volume","target","future_log_ret"]
X = df_final.drop(columns=remove_cols)
X = X.replace([np.inf, -np.inf], np.nan)
mask = X.notna().all(axis=1)
X = X[mask].values
y = df_final["target"]
y = y[mask].values
X.shape

(125501, 28)

### 1st approach
Single TimeSeriesSplit

GridSearch once

Evaluate on last fold

In [ ]:
param_grid = {
    "n_estimators": [500, 1000],
    "max_depth": [5, 7],
    "max_features": ["sqrt"],
    "min_samples_leaf": [5, 10, 20],
}

tscv = TimeSeriesSplit(n_splits=5)

search = GridSearchCV(
    RandomForestClassifier(
        n_jobs=-1,
        random_state=42
    ),
    param_grid,
    cv=tscv,
    scoring="accuracy",
    n_jobs=-1
)

search.fit(X, y)
print(search.best_params_)


{'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 20, 'n_estimators': 500}


#### Evaluation

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)
train_idx, test_idx = list(tscv.split(X))[-1]

X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

final_model = RandomForestClassifier(
    n_estimators=500,
    max_depth=5,
    max_features="sqrt",
    min_samples_leaf=20,
    n_jobs=-1,
    random_state=42,
)

final_model.fit(X_train, y_train)
preds = final_model.predict(X_test)
proba = final_model.predict_proba(X_test)[:, 1]



In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "max_depth": [3, 5, 7],
    "min_samples_leaf": [10, 20, 50]
}

rf = RandomForestClassifier(
    n_estimators=500,
    max_features="sqrt",
    n_jobs=-1,
    random_state=42
)

gs = GridSearchCV(
    rf,
    param_grid,
    cv=5,
    scoring="balanced_accuracy",
    n_jobs=-1
)

gs.fit(X_train, y_train)

best_model = gs.best_estimator_

scoring = {
    "bal_acc": "balanced_accuracy",
    "auc": "roc_auc",
    "logloss": "neg_log_loss"
}

gs = GridSearchCV(
    rf,
    param_grid,
    cv=5,
    scoring=scoring,
    refit="bal_acc",   # ← this chooses the model
    n_jobs=-1
)



Model selection -> Balanced accuracy, reporting -> roc auc log loss economic metrcis

In [27]:
imp = (
    pd.Series(final_model.feature_importances_, index=df_final.drop(columns=remove_cols).columns)
      .sort_values(ascending=False)
)

print(imp.head(10))


sma_ratio_10_50    0.085543
log_ret_20         0.053764
slope_20           0.053569
ema_ratio_5_20     0.052588
slope_10           0.050761
atr_pct            0.046377
gap                0.042105
dist_high_20       0.040432
rsi_14             0.039541
vol_5              0.038558
dtype: float64


In [28]:
from sklearn.metrics import balanced_accuracy_score, roc_auc_score

print("Balanced acc:", balanced_accuracy_score(y_test, preds))
print("ROC AUC:", roc_auc_score(y_test, proba))


from sklearn.metrics import balanced_accuracy_score, roc_auc_score

df_test = df_final.iloc[test_idx]
df_test["pred"] = preds

df_test.groupby("date")["pred"].mean().tail()


Balanced acc: 0.5027966451197711
ROC AUC: 0.5038874808974207


C:\Users\alexd\AppData\Local\Temp\ipykernel_21080\1011101494.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["pred"] = preds


date
2025-11-17    0.586207
2025-11-18    0.655172
2025-11-19    0.758621
2025-11-20    0.758621
2025-11-21    0.620690
Name: pred, dtype: float64

In [29]:
baseline_preds = (df_final.loc[test_idx, "log_ret_1"] > 0).astype(int)
print(
    "Baseline acc:",
    balanced_accuracy_score(y_test, baseline_preds)
)


Baseline acc: 0.4997280468427686


In [30]:
always_up = np.ones_like(y_test)
print(
    "Always-up acc:",
    balanced_accuracy_score(y_test, always_up)
)


Always-up acc: 0.5


### 2nd approach 

Option B (production-grade)

Outer forward walk (evaluation)

Inner CV restricted to outer train only

Panel-aware splitting

Let’s fix this properly.

In [31]:
dates = df_final["date"].values
unique_dates = np.unique(dates)

n_folds = 5
fold_size = len(unique_dates) // n_folds

folds = []
for i in range(n_folds - 1):
    train_dates = unique_dates[: (i + 1) * fold_size]
    test_dates  = unique_dates[(i + 1) * fold_size : (i + 2) * fold_size]

    train_idx = df_final.index[df_final["date"].isin(train_dates)]
    test_idx  = df_final.index[df_final["date"].isin(test_dates)]

    folds.append((train_idx, test_idx))


In [ ]:
inner_tscv = TimeSeriesSplit(n_splits=3)

for train_idx, test_idx in folds:

    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    search = GridSearchCV(
        RandomForestClassifier(
            n_jobs=-1,
            random_state=42
        ),
        param_grid,
        cv=inner_tscv,
        scoring="neg_log_loss",# punish overconfident wrong predictions
        n_jobs=-1
    )

    search.fit(X_train, y_train)

    best_model = search.best_estimator_
    preds = best_model.predict(X_test)


In [ ]:
scoring = "accuracy"

scoring = "balanced_accuracy"
# or
scoring = "roc_auc"
# or
scoring = "neg_log_loss"


In [7]:
# target balance
y.mean()




np.float64(0.5033426028477861)

In [8]:
# per-date leakage check
df_final.groupby("date")["target"].mean().tail()


date
2026-01-28    0.366667
2026-01-29    0.900000
2026-01-30    0.733333
2026-02-02    0.733333
2026-02-03    0.000000
Name: target, dtype: float64

In [9]:

# feature NaNs
X.isna().sum().max()

np.int64(0)

In [2]:
from sklearn.calibration import CalibratedClassifierCV
base_model = RandomForestClassifier(...)

calibrated_model = CalibratedClassifierCV(
    base_model,
    method="isotonic",  # or "sigmoid"
    cv=inner_tscv
)

calibrated_model.fit(X_train, y_train)


NameError: name 'RandomForestClassifier' is not defined

3️⃣ Combining Different Models

Yes — absolutely.

For same horizon (e.g., 1-step ahead):

𝑃
𝑓
𝑖
𝑛
𝑎
𝑙
=
𝑤
1
𝑃
𝑅
𝐹
+
𝑤
2
𝑃
𝑋
𝐺
𝐵
+
𝑤
3
𝑃
𝐿
𝐺
𝐵
P
final
	​

=w
1
	​

P
RF
	​

+w
2
	​

P
XGB
	​

+w
3
	​

P
LGB
	​


This reduces variance.

Even equal weights often improve stability.

4️⃣ Combining Different Horizons (More Interesting)

Now this is more subtle.

You have:

P₁ (1-step)

P₃ (3-step)

P₅ (5-step)

P₇ (7-step)

These capture different time-scale dynamics.

You can:

Option A — Simple Average
𝑃
𝑐
𝑜
𝑚
𝑏
𝑖
𝑛
𝑒
𝑑
=
𝑃
1
+
𝑃
3
+
𝑃
5
+
𝑃
7
4
P
combined
	​

=
4
P
1
	​

+P
3
	​

+P
5
	​

+P
7
	​

	​


This smooths noise.

Option B — Weighted by Validation Performance

Weight by out-of-sample Sharpe or AUC:

𝑃
𝑐
𝑜
𝑚
𝑏
𝑖
𝑛
𝑒
𝑑
=
∑
𝑤
ℎ
𝑃
ℎ
P
combined
	​

=∑w
h
	​

P
h
	​

Option C — Meta Model (Best in Theory)

Train:

Input features:
[P1_RF, P1_XGB, P3_RF, P3_XGB, P5_RF, ...]


Then train a logistic regression on these probabilities.

This learns:

When short horizon dominates

When longer horizon dominates

Regime switching implicitly

This is called stacking.

Very powerful.

5️⃣ Important Concept: H